<h1 style="text-align:center;">Nerve Quadrature Analysis</h1>
<h3 style="text-align:center;margin:0;">Vijay Stroup</h3>
<p style="text-align:center;margin:0;">University of Central Florida • November 6, 2021</p>

# Introduction

In a study conducted by John Cahn, John Mallet-Paret, and Erik van Vleck on 
["Traveling Wave Solutions for Systems of ODEs on a Two-Dimensional Spatial Lattice"](http://www.jstor.org/stable/118432), 
they have deduced an function $\psi$ to represent a impuluse in the human nervious 
system. 

Consider the function $\psi: \mathbb{R}\rightarrow\mathbb{R}$

$$
\psi(x) =
\frac{1}2 + \frac{1}\pi \int_0^\infty \frac{A(s)sin(sx)}{s(A(s)^2 + c^2s^2)} ds +
\frac{c}\pi \int_0^\infty \frac{cos(sx)}{A(s)^2 + c^2s^2} ds
$$

$$ A(s) = 1 + 2(1 - cos(s)) $$

$\psi$ is the wave function representation of We can use different methods of integration to estimate $\psi$ at different 
values of x and c where $c \neq 0$.  

The methods of quadrature approximation that will be used in this project will 
be the following:  
<br/>
Midpoint: $$\int_a^b f(x) dx \approx (b-a)f(\frac{b+a}2)$$  
<br/>
Trapezoidal: $$\int_a^b f(x) dx \approx \frac{b-a}2 (f(a)+f(b)$$  
<br/>
Simpson: $$\int_a^b f(x) dx \approx \frac{h}3 (f(a) + 4f(a+h) + f(a+2h)), \qquad h=\frac{b-a}2$$  
<br/>

In genearl, we can write a quadrature formula as
$$\int_a^b f(x)dx \approx \sum_{i=1}^n w_i f(x_i)$$
where we want to choose weights $w_i$ and nodes so that the formula is as accurate as possible.

We can then use Richardson's error estimate to determine how accurate each of 
the methods in this project are. The reason of using Richardson's error estimate 
is because we do not have any true values to compare to.  
Richardson's error estimate is defined as:
$$|\int_a^b f(x) dx - T_n| \leq \frac{h^2}{12} (b-a) \max_{x \in [a,b]} |f''(x)|$$

I believe that Simpon's Rule will be more accurate when estimaing what this function should be. It will also be able to be more accurate with many less nodes than required by the trapezodial rule. This is shown by calculating the number of nodes needed to achieve a certain error for both methods. While the midpoint rule only requires the two end nodes, it can can be highly inaccurate as we will see later in the error analysis.

In [1]:
from scipy.integrate import trapezoid, simpson, cumulative_trapezoid

f = lambda x: x**2

xs = [0, 1, 2, 3, 4, 5]
ys = [f(i) for i in xs]
print(trapezoid(ys, xs))
print(simpson(ys, xs))
print(cumulative_trapezoid(ys, xs))

42.5
41.83333333333333
[ 0.5  3.   9.5 22.  42.5]


In [17]:
import numpy as np
import matplotlib.pyplot as plt

A = lambda s: 1 + 2 * (1 - np.cos(s))
f1 = lambda c, s, x: (A(s) * np.sin(s*x)) / (s * (A(s)**2 + c**2 * s**2))
f2 = lambda c, s, x: np.cos(s*x) / (A(s)**2 + c**2 * s**2)

c, x = 1, 1
xs = np.linspace(-10, 10, 10)
f1ys = [f1(c, i, x) for i in xs]
f2ys = [f2(c, i, x) for i in xs]
psi = lambda f1, f2: 1/2 + 1/np.pi * simpson(f1) + c/np.pi * simpson(f2)
psis = [psi(f1, f2) for f1, f2 in zip(f1ys, f2ys)]
# plt.plot(xs, psi)

IndexError: tuple index out of range

In [16]:
for x in zip(f1ys, f2ys):
    print(x)

(-0.002088040459580688, -0.006884123348834443)
(0.005321531231576998, 0.0011097127899821289)
(-0.005443242101362567, 0.02253755116320326)
(-0.007938148467594138, -0.027462535709783357)
(0.29905944021995945, 0.07786436936297904)
(0.29905944021995945, 0.07786436936297904)
(-0.007938148467594172, -0.027462535709783343)
(-0.005443242101362546, 0.022537551163203287)
(0.005321531231576999, 0.0011097127899821156)
(-0.002088040459580688, -0.006884123348834443)


In [13]:
f2ys

[-0.006884123348834443,
 0.0011097127899821289,
 0.02253755116320326,
 -0.027462535709783357,
 0.07786436936297904,
 0.07786436936297904,
 -0.027462535709783343,
 0.022537551163203287,
 0.0011097127899821156,
 -0.006884123348834443]